In [26]:
import pandas as pd
import numpy as np

In [27]:
df1 = pd.read_csv('./trade_train.csv')
df_s = pd.read_csv('./stocks.csv')

In [28]:
#해당 데이터를 0과 1사이의 값으로 표준화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_cols = ['매수고객수', '매도고객수', '평균매수수량', 
              '평균매도수량', '매수가격_중앙값', '매도가격_중앙값']
df_scaled = scaler.fit_transform(df1[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    df1[element]=df_scaled[element]
    
scale_cols = ['종목시가', '종목고가', '종목저가', '종목종가', '거래량',
              '거래금액_만원단위']
df_scaled = scaler.fit_transform(df_s[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    df_s[element]=df_scaled[element]

In [29]:
#그룹벡터 생성 코드
temp = df1['그룹번호']
df1.drop('Unnamed: 0',axis=1, inplace=True)
a = df1['그룹번호']
b = []
for element in a:
    b.append(int(element[3:5]))
df1['그룹번호'] = b
temp = np.unique(temp)
temp = temp.reshape(-1,6)
temp2 = temp.tolist()

from gensim.models import Word2Vec
embedding_model = Word2Vec(temp2, size=2, window = 2, 
                           min_count=1, workers=4, iter=100, sg=1)
vec = embedding_model.wv
vocabs = vec.vocab.keys()
word_list = [vec[v] for v in vocabs]
vec = pd.DataFrame()
vec['그룹번호'] = vocabs
for i in range(len(vec)):
    vec.loc[i,'g_vec'] = word_list[i][0]
    vec.loc[i,'g_vec2'] = word_list[i][1]

vec.to_csv('g_vec.csv')

In [31]:
#종목 벡터 생성 코드
df_t = pd.read_csv('./trade_train.csv')
temp = df_t['종목번호']
temp = np.unique(temp).reshape(-1,13).tolist()

from gensim.models import Word2Vec
embedding_model = Word2Vec(temp, size=1, window = 2, 
                           min_count=1, workers=4, iter=100, sg=1)
vec = embedding_model.wv
vocabs = vec.vocab.keys()
# print(vocabs)
word_list = [vec[v] for v in vocabs]
vec = pd.DataFrame()
vec['종목번호'] = vocabs
for i in range(len(vec)):
    vec.loc[i,'s_vec'] = word_list[i][0]
vec.sort_values(by='종목번호',inplace=True)
vec.reset_index(drop=True,inplace=True)
vec.to_csv('s_vec.csv')

In [168]:
#stocks 데이터의 날짜 형식 변경
df_s['기준년월'] = df_s['기준일자'] / 100
df_s = df_s.astype({'기준년월':'int'})
df_s['기준일자'] = df_s['기준일자'] - (df_s['기준년월']*100)
df_s.drop('index',axis=1,inplace=True)

In [169]:
#그룹별로 데이터를 분할하여 배열에 저장
tra_train = []
for i in range(48):
    mask = df1['그룹번호']==i+1
    tra_train.append(df1[mask])
    tra_train[i].reset_index(drop=True,inplace=True)
    
stocks = []
for i in range(48):
    stocks.append(pd.merge(tra_train[i][['기준년월','종목번호']],df_s,
                           on=['기준년월','종목번호']))

In [170]:
#19일치의 데이터만을 사용하기 위한 전처리 과정
df_con=[]
rem = []
for i in range(48):
    df_con.append(pd.DataFrame())
for i in range(48):
    print(i)
    df_group = stocks[i].groupby(['기준년월','종목번호'])
    for key, group in df_group:        
        if len(group) < 19:
            rem.append((i+1,key))
        if len(group) > 18:
            df_con[i] = pd.concat([df_con[i],group.iloc[0:19,:]])
        df_con[i].reset_index(drop=True,inplace=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


In [171]:
#19일 분량의 데이터가 없는 종목들을 월별데이터에서 제거
tra_train2 = tra_train[:]
for i in range(len(rem)):
    mask = tra_train2[rem[i][0]-1]['기준년월'] == rem[i][1][0]
    mask2 = tra_train2[rem[i][0]-1]['종목번호'] == rem[i][1][1]
    idx = tra_train2[rem[i][0]-1][mask2 & mask].index
    tra_train2[rem[i][0]-1] = tra_train2[rem[i][0]-1].drop(idx)
tra_train = tra_train2[:]
for i in range(48):
    tra_train[i].reset_index(drop=True,inplace=True)
for i in range(48):
    mask = tra_train[i]['종목번호']=='A204990'
    idx = tra_train[i][mask].index
    tra_train[i] = tra_train[i].drop(idx)
    tra_train[i].reset_index(drop=True,inplace=True)
stocks = df_con[:]

In [13]:
#검사과정 : 모두 True가 나오면 정상
# count = 0
# count2 = 0
# for i in range(48):
#     df_group = stocks[i].groupby(['기준년월','종목번호'])
#     df_group2 = tra_train[i].groupby(['기준년월','종목번호'])
#     for key, group in df_group:
#         count += 1
#     #     print(1, key)
#     for key, group in df_group2:
#         count2 += 1
#     #     print(2, key)
#     if count==count2:
#         print(count, count2, True)
#     else:
#         print(count, count2, False)
#     count  = 0
#     count2 = 0

249 249 True
107 107 True
529 529 True
1255 1255 True
1412 1412 True
2715 2715 True
96 96 True
963 963 True
1132 1132 True
41 41 True
262 262 True
165 165 True
92 92 True
314 314 True
71 71 True
75 75 True
316 316 True
848 848 True
224 224 True
187 187 True
784 784 True
201 201 True
2373 2373 True
325 325 True
128 128 True
175 175 True
192 192 True
84 84 True
317 317 True
268 268 True
241 241 True
805 805 True
296 296 True
54 54 True
1066 1066 True
385 385 True
668 668 True
1299 1299 True
124 124 True
53 53 True
2984 2984 True
1524 1524 True
2818 2818 True
229 229 True
170 170 True
906 906 True
223 223 True
207 207 True


In [172]:
# 검사과정 : 결과 (Finish)
for i in range(48):
    df_group = stocks[i].groupby(['기준년월','종목번호'])
    for key,group in df_group:
        if len(group) != 19:
            print(False)
print('Finish')

Finish


In [173]:
#월별데이터와 일별 데이터를 순서에 맞게 정렬
for i in range(48):
    stocks[i].sort_values(by=['기준년월','종목번호','기준일자'],inplace = True)
    tra_train[i].sort_values(by=['기준년월','종목번호'],inplace=True)
    stocks[i].reset_index(drop=True,inplace=True)
    tra_train[i].reset_index(drop=True,inplace=True)

In [37]:
#MinMaxScaler를 이용하여 전처리
g_vec = pd.read_csv('g_vec.csv')
s_vec = pd.read_csv('s_vec.csv')

g_vec.drop('Unnamed: 0',axis=1, inplace=True)
s_vec.drop('Unnamed: 0',axis=1, inplace=True)

a = g_vec['그룹번호']
b = []
for element in a:
    b.append(int(element[3:5]))
g_vec['그룹번호'] = b

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_cols = ['g_vec','g_vec2']
df_scaled = scaler.fit_transform(g_vec[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    g_vec[element]=df_scaled[element]

scale_cols = ['s_vec']
df_scaled = scaler.fit_transform(s_vec[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    s_vec[element]=df_scaled[element]

In [175]:
#벡터 데이터와 join 연산 수행
for i in range(48):
    stocks[i]['그룹번호'] = i+1
for i in range(48):
    stocks[i] = pd.merge(stocks[i],g_vec,on=['그룹번호'])
    tra_train[i] = pd.merge(tra_train[i],g_vec,on=['그룹번호'])
for i in range(48):
    stocks[i] = pd.merge(stocks[i],s_vec,on=['종목번호'])
    tra_train[i]=pd.merge(tra_train[i],s_vec,on=['종목번호'])

In [176]:
#월별데이터와 일별 데이터를 순서에 맞게 정렬 및 인덱스 초기화
for i in range(48):
    stocks[i].sort_values(by=['기준년월','종목번호','기준일자'],inplace = True)
    tra_train[i].sort_values(by=['기준년월','종목번호'],inplace=True)
    stocks[i].reset_index(drop=True,inplace=True)
    tra_train[i].reset_index(drop=True,inplace=True)

In [177]:
#배열로 이루어진 데이터를 하나로 모아 csv파일로 저장
df_sto = pd.concat(stocks)
df_tra = pd.concat(tra_train)
df_sto.reset_index(drop=True,inplace=True)
df_tra.reset_index(drop=True,inplace=True)
df_sto.to_csv('train.csv')
df_tra.to_csv('label.csv')

In [178]:
#데이터 불러오기
import pandas as pd
import numpy as np

df_sto = pd.read_csv('train.csv')
df_tra = pd.read_csv('label.csv')
df_big = pd.read_csv('fi_big.csv');

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_cols = ['종가', 'TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator',
       '개인 순매수', '기관 순매수', '외국인 순매수']
df_scaled = scaler.fit_transform(df_big[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    df_big[element]=df_scaled[element]

In [179]:
#join연산을 위해 날짜 형식 변경
date = []
for i in range(len(df_big['날짜'])):
  a = df_big['날짜'][i].replace('-','')
  date.append(a)

df_big['날짜']=date
df_big = df_big.astype({'날짜':'int'})

df_big['기준년월'] = df_big['날짜']/100
df_big = df_big.astype({'기준년월':'int'})
df_big['기준일자'] = df_big['날짜']-df_big['기준년월']*100

In [180]:
#새로운 데이터와 join 연산 수행 및 정렬
df_new = pd.merge(df_sto[['그룹번호','기준년월','기준일자','종목번호','종목명','g_vec','s_vec']],
          df_big[['기준년월','기준일자','종목번호','종가', 'TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator', '개인 순매수', '기관 순매수', '외국인 순매수']],
           on=['기준년월','기준일자','종목번호'])
df_new.sort_values(by=['그룹번호','기준년월','종목번호','기준일자'],inplace = True)

In [181]:
#19일 분량의 데이터가 없는 종목 정보 필터링
rem = []
df_group = df_new.groupby(['그룹번호','기준년월','종목번호'])
for key, group in df_group:
  if len(group) != 19:
    rem.append(key)
    print(key,len(group))

(6, 201912, 'A062970') 2
(8, 201912, 'A053950') 16
(41, 201912, 'A053950') 16
(41, 202003, 'A092190') 15
(42, 201912, 'A053950') 16
(43, 201912, 'A062970') 2
(46, 201912, 'A062970') 2


In [182]:
#해당 종목정보 삭제
df_new2 = df_new.copy()
for element in rem:
   mask = (df_new2['그룹번호']==element[0]) & (df_new2['기준년월']==element[1]) & (df_new2['종목번호']==element[2])
   idx = df_new2[mask].index
   df_new2 = df_new2.drop(idx)

df_tra2 = df_tra.copy()
for element in rem:
  mask = (df_tra2['그룹번호']==element[0]) & (df_tra2['기준년월']==element[1]) & (df_tra2['종목번호']==element[2])
  idx = df_tra2[mask].index
  df_tra2 = df_tra2.drop(idx)

df_new = df_new2
df_tra = df_tra2

filter = (df_tra['기준년월'].isin(df_new['기준년월'])) & (df_tra['종목번호'].isin(df_new['종목번호']))
df_tra2 = df_tra[filter]

In [183]:
#데이터 정렬 및 저장
df_tra = df_tra2
df_new = df_new2

df_tra.sort_values(by=['그룹번호','기준년월','종목번호'],inplace = True)
df_new.sort_values(by=['그룹번호','기준년월','종목번호','기준일자'],inplace = True)

df_new.reset_index(drop=True,inplace=True)
df_tra.reset_index(drop=True,inplace=True)

df_new.to_csv('day_data.csv')
df_tra.to_csv('month_data.csv')

In [22]:
#데이터 불러오기
import numpy as np
import pandas as pd

filepath = ''
df_tra = pd.read_csv(filepath+'label.csv')
df_sto = pd.read_csv(filepath+'train.csv')
d_data = pd.read_csv(filepath+'day_data.csv')
m_data = pd.read_csv(filepath+'month_data.csv')

In [4]:
spi = pd.read_csv(filepath+'종합주가지수.csv')

In [7]:
spi.info()
date = []
for i in range(len(spi['날짜'])):
  a = spi['날짜'][i].replace('-','')
  date.append(a)

spi['날짜']=date
spi = spi.astype({'날짜':'int'})

spi['기준년월'] = spi['날짜']/100
spi = spi.astype({'기준년월':'int'})
spi['기준일자'] = spi['날짜']-spi['기준년월']*100

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      271 non-null    object
 1   KOSPI   271 non-null    object
 2   S&P500  271 non-null    object
dtypes: object(3)
memory usage: 6.5+ KB


In [16]:
for i in range(len(spi['KOSPI'])):
  spi['KOSPI'][i] = spi['KOSPI'][i].replace(',','')
for i in range(len(spi['S&P500'])):
    spi['S&P500'][i] = spi['S&P500'][i].replace(',','')

<ipython-input-16-f4c73f6c11b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spi['KOSPI'][i] = spi['KOSPI'][i].replace(',','')
<ipython-input-16-f4c73f6c11b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spi['S&P500'][i] = spi['S&P500'][i].replace(',','')


In [18]:
spi = spi.astype({'KOSPI':'float','S&P500':'float'})

In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_cols = ['KOSPI','S&P500']
df_scaled = scaler.fit_transform(spi[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    spi[element]=df_scaled[element]

In [23]:
d_data = pd.merge(d_data[['그룹번호', '기준년월', '기준일자', '종목번호', '종목명','g_vec', 's_vec',
       '종가', 'TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator',
       '개인 순매수', '기관 순매수', '외국인 순매수']],spi[['기준년월','기준일자','KOSPI','S&P500']],
                  on=['기준년월','기준일자'])

In [35]:
g_vec = pd.read_csv(filepath+'g_vec.csv') #전처리필요

In [39]:
g_vec

,그룹번호,g_vec,g_vec2
0,1,0.592800,0.809382
1,2,0.127191,0.864075
2,3,0.924270,0.161249
3,4,0.347869,0.340628
4,5,0.704443,0.763284
5,6,0.017142,0.031088
6,7,0.174858,0.290728
7,8,0.335218,0.529945
8,9,0.220643,0.700609
9,10,0.195646,0.984272


In [40]:
d_data = pd.merge(d_data[['그룹번호', '기준년월', '기준일자', '종목번호', '종목명', 's_vec',
       '종가', 'TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator',
       '개인 순매수', '기관 순매수', '외국인 순매수']],g_vec,on=['그룹번호'])

In [28]:
#LSTM을 위한 데이터셋을 만드는 함수
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    j = 0
    for i in range(len(label)):
        feature_list.append(np.array(data.iloc[j:j+window_size]))
        label_list.append(np.array(label.iloc[i]))
        j += window_size
    return np.array(feature_list), np.array(label_list)

In [29]:
#데이터 학습 과정
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

m_data.sort_values(by=['그룹번호','기준년월','종목번호'],inplace = True)
d_data.sort_values(by=['그룹번호','기준년월','종목번호','기준일자'],inplace = True)

train_feature, train_label = make_dataset(d_data[['TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator',
       '개인 순매수','KOSPI','S&P500']],
                                                  m_data['매수고객수'],19)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label,
                                                      test_size=0.2)

model = Sequential()
model.add(LSTM(16, 
                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                activation='relu', 
                return_sequences=False))
model.add(Dense(1,))
model.summary()
model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=15)
filename = filepath+'result.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, 
                              save_best_only=True, mode='auto')
history = model.fit(x_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])    
print('Finish')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 16)                1536      
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 1,553
Trainable params: 1,553
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
1457/1467 [============================>.] - ETA: 0s - loss: 5.1018e-04
Epoch 00001: val_loss improved from inf to 0.00062, saving model to result.h5
1467/1467 [==============================] - 8s 5ms/step - loss: 5.0787e-04 - val_loss: 6.1705e-04
Epoch 2/200
1459/1467 [============================>.] - ETA: 0s - loss: 3.8453e-04
Epoch 00002: val_loss did not improve from 0.00062
1467/1467 [==============================] - 7s 5ms/step - loss: 3.8530e-04 - val_loss: 6.2729e-04
Epoch 3/200
1455/1467

Epoch 31/200
1465/1467 [============================>.] - ETA: 0s - loss: 3.5525e-04
Epoch 00031: val_loss improved from 0.00057 to 0.00056, saving model to result.h5
1467/1467 [==============================] - 7s 5ms/step - loss: 3.5484e-04 - val_loss: 5.6165e-04
Epoch 32/200
1464/1467 [============================>.] - ETA: 0s - loss: 3.5609e-04
Epoch 00032: val_loss did not improve from 0.00056
1467/1467 [==============================] - 7s 5ms/step - loss: 3.5600e-04 - val_loss: 5.7644e-04
Epoch 33/200
1455/1467 [============================>.] - ETA: 0s - loss: 3.5666e-04
Epoch 00033: val_loss did not improve from 0.00056
1467/1467 [==============================] - 8s 5ms/step - loss: 3.5463e-04 - val_loss: 5.7804e-04
Epoch 34/200
1466/1467 [============================>.] - ETA: 0s - loss: 3.5260e-04
Epoch 00034: val_loss did not improve from 0.00056
1467/1467 [==============================] - 7s 5ms/step - loss: 3.5246e-04 - val_loss: 5.6972e-04
Epoch 35/200
1458/1467 [=====

1458/1467 [============================>.] - ETA: 0s - loss: 3.3539e-04
Epoch 00065: val_loss did not improve from 0.00054
1467/1467 [==============================] - 9s 6ms/step - loss: 3.3391e-04 - val_loss: 5.5855e-04
Epoch 66/200
1464/1467 [============================>.] - ETA: 0s - loss: 3.3479e-04
Epoch 00066: val_loss did not improve from 0.00054
1467/1467 [==============================] - 9s 6ms/step - loss: 3.3422e-04 - val_loss: 5.4395e-04
Epoch 67/200
1464/1467 [============================>.] - ETA: 0s - loss: 3.3258e-04
Epoch 00067: val_loss did not improve from 0.00054
1467/1467 [==============================] - 9s 6ms/step - loss: 3.3240e-04 - val_loss: 5.5463e-04
Epoch 68/200
1461/1467 [============================>.] - ETA: 0s - loss: 3.3230e-04
Epoch 00068: val_loss did not improve from 0.00054
1467/1467 [==============================] - 11s 7ms/step - loss: 3.3181e-04 - val_loss: 5.4856e-04
Epoch 69/200
1466/1467 [============================>.] - ETA: 0s - loss

In [30]:
#데이터불러오기
filepath =''
df_big = pd.read_csv(filepath+'fi_big.csv')
df_tra = pd.read_csv(filepath+'month_data.csv')

In [31]:
#날짜 형식 변경
date = []
for i in range(len(df_big['날짜'])):
  a = df_big['날짜'][i].replace('-','')
  date.append(a)

df_big['날짜']=date
df_big = df_big.astype({'날짜':'int'})

df_big['기준년월'] = df_big['날짜']/100
df_big = df_big.astype({'기준년월':'int'})
df_big['기준일자'] = df_big['날짜']-df_big['기준년월']*100

In [32]:
from sklearn.preprocessing import MinMaxScaler
df_s = df_big
scaler = MinMaxScaler()
scale_cols = ['종가', 'TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator', '개인 순매수',
       '기관 순매수', '외국인 순매수']
df_scaled = scaler.fit_transform(df_s[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols
for element in scale_cols:
    df_s[element]=df_scaled[element]

In [33]:
#7월 데이터 필터링
df_s = df_s[df_s.기준년월 == 202007]

In [52]:
df_tra = pd.merge(df_tra[['그룹번호','종목번호', 's_vec']],g_vec,on=['그룹번호'])

In [35]:
df_s = pd.merge(df_s,spi[['기준년월','기준일자','KOSPI','S&P500']],
                on=['기준년월','기준일자'])

In [37]:
df_tra

,Unnamed: 0,Unnamed: 0.1,기준년월,그룹번호,그룹내고객수,종목번호,그룹내_매수여부,그룹내_매도여부,매수고객수,매도고객수,평균매수수량,평균매도수량,매수가격_중앙값,매도가격_중앙값,g_vec,s_vec
0,0,0,201907,1,288,A000660,Y,Y,0.008092,0.028476,0.000052,0.000641,0.052124,0.054666,1.000000,0.629038
1,1,1,201907,1,288,A001820,Y,Y,0.003468,0.006700,0.001460,0.000422,0.033020,0.032522,1.000000,0.406752
2,2,2,201907,1,288,A004020,Y,Y,0.003468,0.005025,0.001121,0.001447,0.028857,0.027856,1.000000,0.245796
3,3,3,201907,1,288,A005380,Y,Y,0.004624,0.005025,0.000885,0.001236,0.097977,0.097931,1.000000,0.820740
4,4,4,201907,1,288,A005930,Y,Y,0.015029,0.031826,0.000093,0.000277,0.031602,0.032626,1.000000,0.274691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29334,29334,29947,202006,48,163,A207940,Y,Y,0.003468,0.006700,0.000008,0.000008,0.581859,0.555014,0.703114,0.570823
29335,29335,29948,202006,48,163,A272210,Y,Y,0.003468,0.005025,0.000247,0.000115,0.006899,0.006178,0.703114,0.547137
29336,29336,29949,202006,48,163,A285130,Y,Y,0.006936,0.006700,0.000743,0.000444,0.076025,0.068246,0.703114,0.618745
29337,29337,29950,202006,48,163,A316140,Y,Y,0.005780,0.006700,0.001058,0.002014,0.007004,0.006909,0.703114,0.849693


In [38]:
#테스트 데이터 생성
test_df=[]
for i in range(48):
    test = df_tra[df_tra.그룹번호 == i+1]
    test.drop_duplicates('종목번호',keep='first',inplace = True)
    df_g1 = pd.merge(test[['그룹번호','종목번호','g_vec','s_vec']],df_s,on='종목번호')
    df_g1.sort_values(by=['기준년월','종목번호','기준일자'],inplace=True)
    df_g1.reset_index(drop=True,inplace=True)

    df_group = df_g1.groupby('종목번호')
    temp = []
    for key, group in df_group :
        temp.append(group.iloc[0:19,:])
    df_g1 = pd.concat(temp,axis=0)
    df_g1 = df_g1.sort_values(by=['기준년월','종목번호','기준일자'])    
    test_df.append(df_g1)

<ipython-input-38-29dba7825a58>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop_duplicates('종목번호',keep='first',inplace = True)


In [56]:
for i in range(48):
  test_df[i] = test_df[i][['그룹번호', '기준년월', '기준일자','종목번호', '종목명', 'g_vec','g_vec2', 's_vec','20년7월TOP3대상여부', '시장구분',
       '대분류', '중분류', '소분류', '종가', 'TEMA', 'CMF', 'MACD Oscillator',
       'Stochastic Oscillator', '개인 순매수', '기관 순매수', '외국인 순매수']]

In [40]:
def make_dataset2(data, window_size):
    feature_list = []
    j = 0
    for i in range(int(len(data)/window_size)):
        feature_list.append(np.array(data.iloc[j:j+window_size]))
        j += window_size
    return np.array(feature_list)

In [41]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model2 = Sequential()
model2.add(LSTM(16,               
                input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                activation='relu', 
                return_sequences=False))
model2.add(Dense(1,))
model2.summary()
model2.compile(loss='mean_squared_error', optimizer='adam')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 16)                1536      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,553
Trainable params: 1,553
Non-trainable params: 0
_________________________________________________________________


In [42]:
model2.load_weights(filepath+'result.h5')
result=[]
for i in range(48):
  test_feature = make_dataset2(test_df[i][['TEMA', 'CMF', 'MACD Oscillator', 'Stochastic Oscillator',
       '개인 순매수','KOSPI','S&P500']],19)
  pred = model2.predict(test_feature)
  final = pd.DataFrame(pred,columns = ['매수고객수'])
  temp = test_df[i].drop_duplicates('종목번호',keep='first')
  temp.reset_index(drop=True,inplace=True)
  temp = temp[['종목번호','종목명','20년7월TOP3대상여부']]
  final = pd.concat([final,temp],axis=1)
  final = final.sort_values(by=['매수고객수','종목번호'],ascending=False)
  final.reset_index(drop=True,inplace=True)
  result.append(final)

In [43]:
count = 0
for i in range(48):
  result[i]['그룹번호']=i+1
  result[i] = result[i][['그룹번호','종목번호','종목명','매수고객수','20년7월TOP3대상여부']]
  temp = result[i].head(3)
  temp = temp[temp['20년7월TOP3대상여부']=='Y']
  count += len(temp)
print(count,3*48, str(count/(3*48)*100)+'%')

113 144 78.47222222222221%


In [63]:
result[1].head(3)

,그룹번호,종목번호,종목명,매수고객수,20년7월TOP3대상여부
0,2,A051910,LG화학,0.010379,Y
1,2,A035720,카카오,0.010010,Y
2,2,A096530,씨젠,0.009253,Y


In [44]:
#결과파일 생성
result_csv = pd.DataFrame(columns=['그룹명','종목번호1','종목번호2','종목번호3'])

for i in range(48):
    temp = []
    temp.append(result[i]['종목번호'][0])
    temp.append(result[i]['종목번호'][1])
    temp.append(result[i]['종목번호'][2])
    temp.sort()
    if i < 9 :
        temp.insert(0,'MAD0'+str(i+1))
    else :
        temp.insert(0,'MAD'+str(i+1))
    result_csv = result_csv.append(pd.Series(temp, index=result_csv.columns),
                                   ignore_index=True)
    
result_csv.to_csv('result.csv')

In [48]:
result[6]

,그룹번호,종목번호,종목명,매수고객수,20년7월TOP3대상여부
0,7,A000660,SK하이닉스,0.012349,Y
1,7,A096770,SK이노베이션,0.010862,Y
2,7,A096530,씨젠,0.010314,Y
3,7,A066570,LG전자,0.008111,N
4,7,A007570,일양약품,0.007872,Y
5,7,A180640,한진칼,0.007476,N
6,7,A010060,OCI,0.007301,N
7,7,A090430,아모레퍼시픽,0.007213,Y
8,7,A010130,고려아연,0.007035,N
9,7,A019170,신풍제약,0.006909,Y


In [45]:
temp_ar = []
for i in range(48):
  temp2 = result[i].head(3)
  temp2 = temp2[temp2['20년7월TOP3대상여부']=='Y']
  temp_ar.append(temp2)
temp2 = pd.concat(temp_ar)
temp2.reset_index(drop=True,inplace=True)
temp2.drop_duplicates('종목번호',keep='first')

,그룹번호,종목번호,종목명,매수고객수,20년7월TOP3대상여부
0,1,A000660,SK하이닉스,0.012349,Y
1,1,A096770,SK이노베이션,0.010862,Y
2,1,A096530,씨젠,0.010314,Y
8,4,A285130,SK케미칼,0.015173,Y
73,31,A001510,SK증권,0.007620,Y
